# Topic Analysis

In [2]:
import re

import pandas as pd
from nltk.corpus import stopwords

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
import en_core_web_sm
nlp = en_core_web_sm.load()

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

stop_words = set(stopwords.words('english'))

In [3]:
data = pd.read_csv("data/q_ethereum_merge_reddit_full10.csv")
data["time"] = pd.to_datetime(data["time"])
data_after = data[data["time"] > "2022-09-15 00:00:00"]
data_during = data[(data["time"] < "2022-09-15 00:00:00") & (data["time"] > "2020-12-01 00:00:00")]
data_before = data[data["time"] < "2020-12-01 00:00:00"]
print("before: " + str(len(data_before)))
print("during: " + str(len(data_during)))
print("after: " + str(len(data_after)))

before: 78
during: 799
after: 145


In [7]:
print(data["text"].apply(lambda x: len(x.split())).mean()) #Average text lengt in words

647.8825831702544


In [3]:
def remove_URL(sample):
    return re.sub(r"http\S+", "", sample)

def sub_abb(sample):
    return re.sub("eth", "ethereum", sample)

def tokenizer(texts):
    for text in texts:
        text = remove_URL(text)
        text = sub_abb(text)
        yield simple_preprocess(text, deacc=True)


tokens_list = list(tokenizer(data['text']))
bigram = gensim.models.Phrases(tokens_list, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[tokens_list], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    texts_out = []
    for doc in texts:
        joined = nlp(" ".join(doc))
        texts_out.append([token.lemma_ for token in joined if token.pos_ in allowed_postags])
    return texts_out

In [4]:
tokens_list = remove_stopwords(tokens_list)
tokens_list = make_bigrams(tokens_list)
tokens_list = make_trigrams(tokens_list)
lemmas = lemmatization(tokens_list)

# Create Dictionary
dictionary = corpora.Dictionary(lemmas)
# Create Corpus, i.e. Document-Term Matrix
corpus = [dictionary.doc2bow(text) for text in lemmas]

In [5]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=4,
                                           random_state=1,
                                           eval_every = 20,
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=False)
lda_viz = gensimvis.prepare(lda_model, corpus, dictionary)
lda_viz

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.063679 -0.152673       1        1  41.537902
2     -0.017087  0.019268       2        1  30.963876
1     -0.208081  0.031096       3        1  16.344641
0      0.161490  0.102309       4        1  11.153581, topic_info=             Term         Freq        Total Category  logprob  loglift
52       ethereum  3866.000000  3866.000000  Default  30.0000  30.0000
258       bitcoin  1217.000000  1217.000000  Default  29.0000  29.0000
116           nft   619.000000   619.000000  Default  28.0000  28.0000
1543     dogecoin   860.000000   860.000000  Default  27.0000  27.0000
242   transaction  1901.000000  1901.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
208           day    98.100041   572.430070   Topic4  -5.3638   0.4295
129        people   120.045602  1239.588787   Topic4  -5.1619  -0.1413
99         market   103.946840   770.268300   Topic4  -5.3059   0.1906
545         datum    84.625165   447.144069   Topic4  -5.5116   0.5288
113           new   102.024300  1218.886638   Topic4  -5.3246  -0.2871

[332 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2188      1  0.931885    ability
2188      2  0.009227    ability
2188      3  0.036906    ability
2188      4  0.023066    ability
249       1  0.394511       able
...     ...       ...        ...
502       2  0.639438       year
502       3  0.021203       year
502       4  0.072537       year
5766      1  0.985652      zcash
5061      1  0.985609  zk_rollup

[637 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

## Before

In [6]:
tokens_before = list(tokenizer(data_before['text']))
tokens_before = remove_stopwords(tokens_before)
tokens_before = make_bigrams(tokens_before)
tokens_before = make_trigrams(tokens_before)
lemmas_before = lemmatization(tokens_before)

# Create Dictionary
dictionary_before = corpora.Dictionary(lemmas_before)
# Create Corpus, i.e. Document-Term Matrix
corpus_before = [dictionary_before.doc2bow(text) for text in lemmas_before]

lda_model_before = gensim.models.ldamodel.LdaModel(corpus=corpus_before,
                                           id2word=dictionary_before,
                                           num_topics=4,
                                           random_state=100,
                                           eval_every = 20,
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_viz_before = gensimvis.prepare(lda_model_before, corpus_before, dictionary_before)
lda_viz_before

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.151758 -0.000726       1        1  28.607181
0      0.083381 -0.065427       2        1  26.013333
3      0.009799 -0.030137       3        1  24.421281
2      0.058579  0.096289       4        1  20.958205, topic_info=             Term        Freq       Total Category  logprob  loglift
504         block  385.000000  385.000000  Default  30.0000  30.0000
3077        nexus   66.000000   66.000000  Default  29.0000  29.0000
502       bitcoin  342.000000  342.000000  Default  28.0000  28.0000
390      platform   92.000000   92.000000  Default  27.0000  27.0000
19          datum  112.000000  112.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
64           node   25.391810  122.765238   Topic4  -5.3577  -0.0132
4     application   19.394299   57.187491   Topic4  -5.6271   0.4813
386        people   21.436434   92.057663   Topic4  -5.5270   0.1053
62            new   22.860893  188.934070   Topic4  -5.4627  -0.5493
125          user   19.437956  106.421190   Topic4  -5.6249  -0.1375

[302 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
196       1  0.039284      able
196       2  0.353556      able
196       3  0.216062      able
196       4  0.392840      able
135       1  0.029870    access
...     ...       ...       ...
3019      3  0.859775  wireless
132       1  0.338456      work
132       2  0.299403      work
132       3  0.208280      work
132       4  0.156210      work

[533 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

## During

In [7]:
tokens_during = list(tokenizer(data_during['text']))
tokens_during = remove_stopwords(tokens_during)
tokens_during = make_bigrams(tokens_during)
tokens_during = make_trigrams(tokens_during)
lemmas_during = lemmatization(tokens_during)

# Create Dictionary
dictionary_during = corpora.Dictionary(lemmas_during)
# Create Corpus, i.e. Document-Term Matrix
corpus_during = [dictionary_during.doc2bow(text) for text in lemmas_during]

lda_model_during = gensim.models.ldamodel.LdaModel(corpus=corpus_during,
                                           id2word=dictionary_during,
                                           num_topics=6,
                                           random_state=100,
                                           eval_every = 20,
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_viz_during = gensimvis.prepare(lda_model_during, corpus_during, dictionary_during)
lda_viz_during

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.090615 -0.144983       1        1  41.861343
4     -0.109179  0.019039       2        1  25.068788
0     -0.139997  0.154891       3        1  14.909813
3      0.029615 -0.118508       4        1  11.308382
5      0.125808  0.040593       5        1   3.931935
1      0.184368  0.048967       6        1   2.919739, topic_info=          Term         Freq        Total Category  logprob  loglift
2902  dogecoin   801.000000   801.000000  Default  30.0000  30.0000
21         eth  1828.000000  1828.000000  Default  29.0000  29.0000
4     ethereum  3435.000000  3435.000000  Default  28.0000  28.0000
181        nft   757.000000   757.000000  Default  27.0000  27.0000
1149    rollup   709.000000   709.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
74        time    20.312954  1037.798082   Topic6  -5.3290  -0.3999
342      asset    17.589387   422.852036   Topic6  -5.4729   0.3539
180        new    17.315027   881.872922   Topic6  -5.4886  -0.3968
970      datum    16.551367   288.416039   Topic6  -5.5337   0.6757
4     ethereum    15.371611  3435.202862   Topic6  -5.6077  -1.8756

[466 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
218       1  0.410656       able
218       2  0.428057       able
218       3  0.076563       able
218       4  0.069603       able
218       5  0.006960       able
...     ...       ...        ...
456       5  0.042608       year
80        3  0.140754        zil
80        6  0.844521        zil
6800      6  0.949545  zookeeper
1232      5  0.968745         zu

[1015 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 6, 2])

## After

In [8]:
tokens_after = list(tokenizer(data_after['text']))
tokens_after = remove_stopwords(tokens_after)
tokens_after = make_bigrams(tokens_after)
tokens_after = make_trigrams(tokens_after)
lemmas_after = lemmatization(tokens_after)

# Create Dictionary
dictionary_after = corpora.Dictionary(lemmas_after)
# Create Corpus, i.e. Document-Term Matrix
corpus_after = [dictionary_after.doc2bow(text) for text in lemmas_after]

lda_model_after = gensim.models.ldamodel.LdaModel(corpus=corpus_after,
                                           id2word=dictionary_after,
                                           num_topics=6,
                                           random_state=100,
                                           eval_every = 20,
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_viz_after = gensimvis.prepare(lda_model_after, corpus_after, dictionary_after)
lda_viz_after

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.009273 -0.045876       1        1  24.692260
2     -0.002693 -0.124685       2        1  22.320911
1     -0.043416 -0.002001       3        1  21.896729
0     -0.146472  0.076495       4        1  14.857347
4      0.117171  0.095103       5        1   9.662289
3      0.066138  0.000965       6        1   6.570464, topic_info=          Term        Freq       Total Category  logprob  loglift
52    ethereum  298.000000  298.000000  Default  30.0000  30.0000
99      market  110.000000  110.000000  Default  29.0000  29.0000
406    address   78.000000   78.000000  Default  28.0000  28.0000
522      block   80.000000   80.000000  Default  27.0000  27.0000
51         eth  167.000000  167.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
46   ecosystem    4.197216   41.058259   Topic6  -5.6757   0.4420
298       good    4.183643   51.076118   Topic6  -5.6790   0.2205
196        web    4.222731   60.994312   Topic6  -5.6697   0.0523
110      month    4.053937   63.379743   Topic6  -5.7105  -0.0269
565        fee    3.803070   72.497254   Topic6  -5.7743  -0.2252

[420 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2539      2  0.889985  acquisition
676       2  0.931867     activate
406       1  0.102544      address
406       2  0.704990      address
406       3  0.038454      address
...     ...       ...          ...
502       3  0.284071         year
502       4  0.137977         year
502       5  0.040582         year
502       6  0.032465         year
1248      4  0.968251         zone

[878 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 2, 1, 5, 4])